In [ ]:
just_checking_integrity=False
rows=3000
test_rows=1000

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import numpy as np
import datetime
from itertools import compress
from math import sin, cos
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

## Custom metric


In [ ]:
def mrr_top_at(at=100):
    @tf.function
    def mrr_top(y_true,y_pred):
        top_k=tf.math.top_k(y_pred,k=at)
        rr=top_k.indices
        #print(rr)
        idx=tf.range(start=1,limit=at+1,delta=1)
        y_true=tf.reshape(y_true,[-1,1])
        #print(y_true)
        #y_true=tf.broadcast_to(y_true,test.shape)
        y_true=tf.cast(y_true,tf.int32)
        ranking=tf.where(tf.math.equal(rr,y_true),idx,0)
        #print(ranking)
        ranking=tf.reduce_sum(ranking,axis=-1)
        ranking=tf.where(ranking>0,1/ranking,0)
        #print(ranking)
        ranking=tf.reduce_mean(ranking)
        return ranking
    return mrr_top


## Custom layers

In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(
        self,
        maxlen,
        vocab_size,
        embed_dim,
        item_embedding_trainable=True,
        embedding_weights=None
        ):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(
            input_dim=vocab_size,
            output_dim=embed_dim,
            trainable=item_embedding_trainable,
            weights=embedding_weights
            )
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=maxlen-1, limit=0, delta=-1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
class PositionEmbedding(layers.Layer):
    def __init__(
        self,
        maxlen,
        embed_dim,
        ):
        super(PositionEmbedding, self).__init__()
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        return positions

In [ ]:
class RBFLayer(layers.Layer):
    def __init__(self, weights, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        # shape=(embedding_dimension, num_units)
        self.embedding_weights = tf.transpose(weights)
        self.gamma = gamma

    def call(self, inputs):
        l2 = tf.math.reduce_sum(
            tf.math.squared_difference(
                x=tf.expand_dims(inputs, axis=-1),
                y=self.embedding_weights
                ),
            axis=1
            )
        res = tf.math.exp(-self.gamma * l2)
        return res / tf.reduce_sum(res, axis=1, keepdims=True)

# Prepare dataset

In [ ]:
candidate_items = pd.read_csv('./drive/MyDrive/Recommendation_system/dataset/processed_data/candidate_items_mapped.csv')
candidate_items.head()

,item_id
0,1
1,2
2,3
3,4
4,5


In [ ]:
embedding_weights = np.load(
    './drive/MyDrive/Recommendation_system/dataset/processed_data/compressed_features.npy'
    )
import os
import scipy.sparse as sps
def get_ICM(files_directory="./drive/MyDrive/Recommendation_system/dataset/processed_data"):
    df_icm = pd.read_csv(filepath_or_buffer=os.path.join(files_directory, 'simplified_features_and_categories_30.csv'), sep=',', header=0)

    item_id_list = df_icm['item_id'].values
    feat_id_list = df_icm['feature_idx'].values
    rating_id_list = np.ones_like(feat_id_list)
    ICM_matrix = sps.csr_matrix((rating_id_list, (item_id_list, feat_id_list)))
    return ICM_matrix

ICM=get_ICM()
embedding_weights.shape

(23692, 64)

In [ ]:
result =  pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN.csv",parse_dates=["date_session_starting"],
     infer_datetime_format=True)



result["item_id"]=result["item_id"].map(ast.literal_eval)
result["timedelta"]=result["timedelta"].map(ast.literal_eval)

def is_reseen(x):
    #print(x)
    seen={}
    res=[]
    for item in x:
        if item in seen:
            res.append(seen[item])
            seen[item]+=1
        else:
            res.append(0)
            seen[item]=1
    return res

def percentage_seen(x):
    size=len(x)
    return [(i+1)/size for i in range(size)]

In [ ]:
x_reseen = tf.keras.preprocessing.sequence.pad_sequences(
    result['item_id'].progress_apply(is_reseen),
    padding='post',
    maxlen=100,
    dtype="int16"
)
x_percentage_seen = tf.keras.preprocessing.sequence.pad_sequences(
    result['item_id'].progress_apply(percentage_seen),
    padding='post',
    maxlen=100,
    dtype='float16',
)

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
def similarity_with_preceding(x):
    if len(x)==1:
        return np.array([0])
    feats=embedding_weights[x]
    #normalize
    feats=np.divide(feats,np.linalg.norm(feats,axis=1).reshape((-1,1)))
    feats_copy=feats.copy()
    feats_copy=np.roll(feats_copy,1,axis=0)
    feats_copy[0,:]=0
    prod=np.multiply(feats,feats_copy).sum(axis=1)
    return prod



def similarity_feats_with_preceding(x):
    if len(x)==1:
        return np.array([0])
    feats=ICM[x].toarray()
    #normalize
    feats=np.divide(feats,np.linalg.norm(feats,axis=1).reshape((-1,1)))
    feats_copy=feats.copy()
    feats_copy=np.roll(feats_copy,1,axis=0)
    feats_copy[0,:]=0
    prod=np.multiply(feats,feats_copy).sum(axis=1)
    return prod

In [ ]:
x_simils = tf.keras.preprocessing.sequence.pad_sequences(
    result['item_id'].progress_apply(similarity_with_preceding
    ),
    padding='post',
    maxlen=100,
    dtype='float16',
)
x_simils_feats = tf.keras.preprocessing.sequence.pad_sequences(
    result['item_id'].progress_apply(similarity_feats_with_preceding
    ),
    padding='post',
    maxlen=100,
    dtype='float16',
)

  0%|          | 0/1000000 [00:00<?, ?it/s]

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
static_features = [
                   'date_hour_sin',
                   'date_hour_cos',
                   'date_day_sin',
                   'date_day_cos',
                   'date_month_sin',
                   'date_month_cos',
                   'date_hour_sin_ending',
                   'date_hour_cos_ending',
                   'date_day_sin_ending',
                   'date_day_cos_ending',
                   'date_month_sin_ending',
                   'date_month_cos_ending',
                   'date_year_2020',
                   'length_of_session_seconds',
                   'avg_time_spent_per_item_seconds',
                   'variance_time_spent_per_item_seconds',
                   'n_seen_items',
                   'n_unique_items',
                   'user_went_afk',
                   'is_weekend',
                   'is_hot_hour',
                   'is_night',
                   'is_christmas_time',
                   'is_black_friday',
                   'session_similarity',
                   'session_similarity_uniques',
                   'session_similarity_feats',
                   'session_similarity_feats_uniques'
]
for col in ['length_of_session_seconds','avg_time_spent_per_item_seconds','variance_time_spent_per_item_seconds']:
    result[col]=np.log10(result[col]+1)
for col in ['n_seen_items','n_unique_items']:
    result[col]=result[col]/100
item_related_features = [
                         'timedelta'
]

In [ ]:
if just_checking_integrity:
    train_purchases = pd.read_csv(
    './drive/MyDrive/Recommendation_system/dataset/processed_data/train_purchases_mapped.csv',
    usecols = ['session_id', 'item_id'],
    nrows=len(result)
    )
else:
    train_purchases = pd.read_csv(
    './drive/MyDrive/Recommendation_system/dataset/processed_data/train_purchases_mapped.csv',
    usecols = ['session_id', 'item_id'],
    )


In [ ]:
train_purchases.sort_values(by="session_id",inplace=True)

In [ ]:
x_ids = tf.keras.preprocessing.sequence.pad_sequences(
    result['item_id'],
    padding='post',
    maxlen=100
)

In [ ]:
for item_related_feature in item_related_features:
  print('Padding {}'.format(item_related_feature))
  x_item_related = tf.keras.preprocessing.sequence.pad_sequences(
      result[item_related_feature],
      dtype='float16',
      maxlen=100,
      padding='post'
    )

Padding timedelta


In [ ]:
if len(item_related_features) == 1:
    x_item_related = np.expand_dims(x_item_related, axis=-1)

In [ ]:
y = train_purchases['item_id'].to_numpy()

In [ ]:
y_features = embedding_weights[y]

In [ ]:
x_ids_train, x_ids_test,\
x_reseen_train, x_reseen_test,\
x_simils_train,x_simils_test,\
x_simils_feats_train,x_simils_feats_test,\
x_percentage_seen_train,x_percentage_seen_test,\
x_static_train, x_static_test, \
x_item_related_train, x_item_related_test, \
y_train, y_test, \
y_features_train, y_features_test = train_test_split(
    x_ids,
    x_reseen,
    x_simils,
    x_simils_feats,
    x_percentage_seen,
    result[static_features].to_numpy(),
    x_item_related,
    y,
    y_features,
    test_size=0.2,
    random_state=1234
    )


In [ ]:
del x_ids,x_reseen,x_simils,x_simils_feats,x_percentage_seen,result,x_item_related,y,y_features
import gc
gc.collect()

36

# Build Dataset

In [ ]:

train_set_complete = tf.data.Dataset.from_tensor_slices(
    ((x_ids_train, x_reseen_train/100,x_percentage_seen_train,x_simils_train,x_simils_feats_train, x_item_related_train, x_static_train), (y_train, y_features_train))
    ).batch(256,num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE).shuffle(1563, reshuffle_each_iteration=True)
test_set_complete = tf.data.Dataset.from_tensor_slices(
    ((x_ids_test, x_reseen_test/100, x_percentage_seen_test,x_simils_test,x_simils_feats_test, x_item_related_test, x_static_test), (y_test, y_features_test))
    ).batch(256)
gc.collect()

0

In [ ]:
shape=y_features_train.shape[1]

In [ ]:
del x_ids_train,x_reseen_train,x_simils_train,x_simils_feats_train,x_percentage_seen_train,x_item_related_train,y_train,y_features_train
del x_ids_test,x_reseen_test,x_simils_test,x_simils_feats_test,x_percentage_seen_test,x_item_related_test,y_test,y_features_test
gc.collect()

0

In [ ]:
train_set_complete.element_spec

((TensorSpec(shape=(None, 100), dtype=tf.int32, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float64, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float16, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float16, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float16, name=None),
  TensorSpec(shape=(None, 100, 1), dtype=tf.float16, name=None),
  TensorSpec(shape=(None, 28), dtype=tf.float64, name=None)),
 (TensorSpec(shape=(None,), dtype=tf.int64, name=None),
  TensorSpec(shape=(None, 64), dtype=tf.float32, name=None)))

In [ ]:
embed_dim = 64  # Embedding size for each token
maxlen = 100
num_static = len(static_features)
num_item_related = len(item_related_features)
vocabulary_size = 23692 #max(train_purchases['item_id']) # Size of the vocabulary
num_heads = 3
ff_dim = 128
transformer_width = 128 - num_static - num_item_related-4
mlp_width = 256
mlp_depth = 2
###### INPUT ########### INPUT #####
id_inputs = layers.Input(shape=(maxlen,), name='ids')
item_related_inputs = layers.Input(shape=(maxlen, num_item_related), name='item_related_features')
x_reseen_inputs = layers.Input(shape=(maxlen, 1), name='x_reseen_featurs')
x_percentage_seen_inputs = layers.Input(shape=(maxlen, 1), name='x_percentage_seen')
x_simils_inputs = layers.Input(shape=(maxlen, 1), name='x_similarities_embeddings')
x_simils_feats_inputs = layers.Input(shape=(maxlen, 1), name='x_similarities_ICM')
static_inputs = layers.Input(shape=(num_static,), name='static_features')


##### MODEL DEFINITION #####

learned_embedding = layers.Embedding(
    input_dim=vocabulary_size,
    output_dim=embed_dim,
    mask_zero=True,
    name='learned_embedding'
    )(id_inputs)

feature_embedding = layers.Embedding(
    input_dim=embedding_weights.shape[0],
    output_dim=embedding_weights.shape[1],
    mask_zero=True,
    trainable=False,
    weights=[embedding_weights],
    name='feature_embedding'
    )(id_inputs)

positional_embedding = PositionEmbedding(
    maxlen,
    embed_dim=embed_dim*2+num_item_related+4
    )(id_inputs)

embedding = layers.Concatenate(
    name='embedding'
)([learned_embedding, feature_embedding])

masked_item_related_inputs = item_related_inputs[:, :tf.shape(id_inputs)[-1], :]

embedding = layers.Concatenate()([embedding, masked_item_related_inputs, x_reseen_inputs,x_percentage_seen_inputs,x_simils_inputs,x_simils_feats_inputs])

x_pre = embedding + positional_embedding

x_post = TransformerBlock(
    embed_dim*2+num_item_related+4,
    num_heads,
    ff_dim
    )(x_pre)

x_post = layers.Concatenate()([x_post, x_pre])
x_post = layers.GlobalAveragePooling1D()(x_post)

x_post = layers.Dense(
    units=mlp_width-num_static,
    activation='linear'
    )(x_post)

x_pre = layers.Concatenate()([x_post, static_inputs])

for layer in range(mlp_depth):
  x_post = layers.LayerNormalization()(x_pre)
  x_post = layers.Dropout(0.2)(x_post)
  x_post = layers.Dense(units=mlp_width, activation='gelu')(x_post)
  x_pre = layers.Add()([x_pre, x_post])

##### PREDICTION DEFINITION #####

outputs = layers.Dense(19021, activation='softmax', name='y')(x_pre)
secondary_outputs = layers.Dense(
    units=shape,
    activation='tanh',
    name='y_features'
    )(x_pre)

model_for_training= tf.keras.Model(
    inputs=[id_inputs, x_reseen_inputs, x_percentage_seen_inputs,x_simils_inputs,x_simils_feats_inputs,item_related_inputs, static_inputs],
    outputs=[outputs, secondary_outputs],
    name="encodermodel_training"
    )


model_for_training.compile(
    optimizer=tf.keras.optimizers.Adamax(

    ),
    loss=[
          lambda y_true, y_pred : tf.keras.losses.categorical_crossentropy(
              tf.squeeze(tf.one_hot(tf.cast(y_true, dtype=tf.int32), 19020+1, axis=-1)),
              y_pred,
              label_smoothing=0.1
              ),
          'mse'
    ],
    loss_weights=[1, 1],
    metrics=[
             [
              mrr_top_at(at=100)
             ],
             []
             ]
    )



# Plot

#Training

In [ ]:
gc.collect()
if just_checking_integrity:
    n_epochs=2
else:
    n_epochs=100
monitor='val_y_mrr_top'
model_for_training.fit(
    train_set_complete,
    validation_data=test_set_complete,
    epochs=n_epochs,
    callbacks=[
               tf.keras.callbacks.EarlyStopping(
                  monitor=monitor,
                  mode='max',
                  patience=1,
                  min_delta=0.0001,
                  restore_best_weights=True,
              ),
         ]
    )

Epoch 1/100
3125/3125 [==============================] - 254s 77ms/step - loss: 8.2629 - y_loss: 8.2367 - y_features_loss: 0.0262 - y_mrr_top: 0.0553 - val_loss: 7.5383 - val_y_loss: 7.5250 - val_y_features_loss: 0.0133 - val_y_mrr_top: 0.0932
Epoch 2/100
3125/3125 [==============================] - 207s 65ms/step - loss: 7.2802 - y_loss: 7.2668 - y_features_loss: 0.0134 - y_mrr_top: 0.1136 - val_loss: 7.1536 - val_y_loss: 7.1417 - val_y_features_loss: 0.0118 - val_y_mrr_top: 0.1268
Epoch 3/100
3125/3125 [==============================] - 206s 65ms/step - loss: 6.9722 - y_loss: 6.9600 - y_features_loss: 0.0122 - y_mrr_top: 0.1403 - val_loss: 6.9871 - val_y_loss: 6.9756 - val_y_features_loss: 0.0115 - val_y_mrr_top: 0.1449
Epoch 4/100
3125/3125 [==============================] - 206s 65ms/step - loss: 6.7855 - y_loss: 6.7737 - y_features_loss: 0.0118 - y_mrr_top: 0.1581 - val_loss: 6.9029 - val_y_loss: 6.8916 - val_y_features_loss: 0.0114 - val_y_mrr_top: 0.1548
Epoch 5/100
3125/3125 [=

# Prepare test predictions Leaderboard



In [ ]:
del train_set_complete
del test_set_complete

In [ ]:
temp=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN_leaderboard.csv")
temp["item_id"]=temp["item_id"].map(ast.literal_eval)
temp["timedelta"]=temp["timedelta"].map(ast.literal_eval)

test_leaderboard_sessions=temp
gc.collect()

462

In [ ]:

x_reseen_test = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'].apply(is_reseen),
    padding='post',
    maxlen=100,
)

x_percentage_seen_test = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'].apply(percentage_seen),
    padding='post',
    maxlen=100,
)

In [ ]:
x_simils_test = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'].progress_apply(similarity_with_preceding
    ),
    padding='post',
    maxlen=100,
    dtype='float32',
)
x_simils_feats_test = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'].progress_apply(similarity_feats_with_preceding
    ),
    padding='post',
    maxlen=100,
    dtype='float32',
)

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
for col in ['length_of_session_seconds','avg_time_spent_per_item_seconds','variance_time_spent_per_item_seconds']:
    test_leaderboard_sessions[col]=np.log10(test_leaderboard_sessions[col]+1)
for col in ['n_seen_items','n_unique_items']:
    test_leaderboard_sessions[col]=test_leaderboard_sessions[col]/100

In [ ]:
x_test_leaderboard = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'],
    padding='post',
    maxlen=100
)

x_test_leaderboard[0]

array([3404,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)

In [ ]:
for item_related_feature in item_related_features:
  print('Padding {}'.format(item_related_feature))
  x_test_item_related = tf.keras.preprocessing.sequence.pad_sequences(
      test_leaderboard_sessions[item_related_feature],
      dtype='float16',
      maxlen=100,
      padding='post'
    )

Padding timedelta


In [ ]:
if len(item_related_features) == 1:
    x_test_item_related = np.expand_dims(x_test_item_related, axis=-1)

In [ ]:
submission_set = tf.data.Dataset.from_tensor_slices(
    (
        test_leaderboard_sessions['session_id'],
     (
        x_test_leaderboard,
        x_reseen_test/100,
        x_percentage_seen_test,
        x_simils_test,
        x_simils_feats_test,
        x_test_item_related,
        test_leaderboard_sessions[static_features].to_numpy()
     )
    )
).batch(512)

submission_set.element_spec

(TensorSpec(shape=(None,), dtype=tf.int64, name=None),
 (TensorSpec(shape=(None, 100), dtype=tf.int32, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float64, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.int32, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 100, 1), dtype=tf.float16, name=None),
  TensorSpec(shape=(None, 28), dtype=tf.float64, name=None)))

In [ ]:
submission_df = pd.DataFrame(
    {
        'session_id':[],
        'code':[],
        'score':[],
        'rank':[]
    }
)

submission_df.head()

,session_id,code,score,rank


In [ ]:
candidates=np.array([i+1 for i in range(4990)]) # this is for the test month

In [ ]:
for session_ids, sessions in submission_set:
    predicted_scores=model_for_training(sessions)[0].numpy()
    mask = np.ones(predicted_scores.shape[1], dtype=bool)
    mask[candidates] = False
    predicted_scores[...,mask]=-np.inf
    predicted_scores=tf.convert_to_tensor(predicted_scores)
    preds=tf.math.top_k(predicted_scores, k=100)
    scores=preds.values.numpy()
    code = preds.indices.numpy()
    prediction = pd.DataFrame(
        {
            'session_id':session_ids,
            'code':[tuple(v) for v in code],
            'score':[tuple(v) for v in scores],
            'rank':[tuple([rank for rank in range(1, 101)]) for id in session_ids]
        }
        )

    submission_df = pd.concat([submission_df, prediction])

submission_df.head()

,session_id,code,score,rank
0,26.0,"(4952, 4685, 606, 3646, 1205, 1337, 1219, 3894...","(0.10522806, 0.08723683, 0.075930975, 0.065088...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,200.0,"(1827, 772, 3400, 3893, 1468, 64, 200, 4878, 4...","(0.17791617, 0.113633245, 0.032776244, 0.02053...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,205.0,"(4986, 3398, 1484, 1181, 2289, 2235, 724, 737,...","(0.2811312, 0.22937286, 0.030350123, 0.0302500...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,495.0,"(3815, 4845, 4340, 4900, 534, 4562, 3195, 2022...","(0.035377834, 0.020160548, 0.015763562, 0.0141...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,521.0,"(4819, 2121, 1768, 1004, 2232, 1356, 410, 2915...","(0.021545004, 0.015203325, 0.014995563, 0.0138...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
submission_df.to_csv(
    './drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/leaderboard/candidates.csv',
    index=False
    )

In [ ]:
del submission_df
gc.collect()

49

# Prepare test predictions Final


In [ ]:
temp=pd.read_csv("./drive/MyDrive/Recommendation_system/dataset/processed_data/macro_feats_NN_final.csv")
temp["item_id"]=temp["item_id"].map(ast.literal_eval)
temp["timedelta"]=temp["timedelta"].map(ast.literal_eval)

test_leaderboard_sessions=temp
gc.collect()

2035

In [ ]:

x_reseen_test = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'].apply(is_reseen),
    padding='post',
    maxlen=100,
)

x_percentage_seen_test = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'].apply(percentage_seen),
    padding='post',
    maxlen=100,
)

In [ ]:
x_simils_test = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'].progress_apply(similarity_with_preceding
    ),
    padding='post',
    maxlen=100,
    dtype='float32',
)
x_simils_feats_test = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'].progress_apply(similarity_feats_with_preceding
    ),
    padding='post',
    maxlen=100,
    dtype='float32',
)

  0%|          | 0/50000 [00:00<?, ?it/s]

  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
for col in ['length_of_session_seconds','avg_time_spent_per_item_seconds','variance_time_spent_per_item_seconds']:
    test_leaderboard_sessions[col]=np.log10(test_leaderboard_sessions[col]+1)
for col in ['n_seen_items','n_unique_items']:
    test_leaderboard_sessions[col]=test_leaderboard_sessions[col]/100

In [ ]:
x_test_leaderboard = tf.keras.preprocessing.sequence.pad_sequences(
    test_leaderboard_sessions['item_id'],
    padding='post',
    maxlen=100
)

x_test_leaderboard[0]

array([4785,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)

In [ ]:

for item_related_feature in item_related_features:
    print('Padding {}'.format(item_related_feature))
    x_test_item_related = tf.keras.preprocessing.sequence.pad_sequences(
        test_leaderboard_sessions[item_related_feature],
        dtype='float16',
        maxlen=100,
        padding='post'
    )

Padding timedelta


In [ ]:
if len(item_related_features) == 1:
    x_test_item_related = np.expand_dims(x_test_item_related, axis=-1)

In [ ]:
submission_set = tf.data.Dataset.from_tensor_slices(
    (
        test_leaderboard_sessions['session_id'],
     (
        x_test_leaderboard,
        x_reseen_test/100,
        x_percentage_seen_test,
        x_simils_test,
        x_simils_feats_test,
        x_test_item_related,
        test_leaderboard_sessions[static_features].to_numpy()
     )
    )
).batch(512)
submission_set.element_spec

(TensorSpec(shape=(None,), dtype=tf.int64, name=None),
 (TensorSpec(shape=(None, 100), dtype=tf.int32, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float64, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.int32, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 100), dtype=tf.float32, name=None),
  TensorSpec(shape=(None, 100, 1), dtype=tf.float16, name=None),
  TensorSpec(shape=(None, 28), dtype=tf.float64, name=None)))

In [ ]:
submission_df = pd.DataFrame(
    {
        'session_id':[],
        'code':[],
        'score':[],
        'rank':[]
    }
)

submission_df.head()

,session_id,code,score,rank


In [ ]:
candidates=np.array([i+1 for i in range(4990)])

In [ ]:
for session_ids, sessions in submission_set:
    predicted_scores=model_for_training(sessions)[0].numpy()
    mask = np.ones(predicted_scores.shape[1], dtype=bool)
    mask[candidates] = False
    predicted_scores[...,mask]=-np.inf
    predicted_scores=tf.convert_to_tensor(predicted_scores)
    preds=tf.math.top_k(predicted_scores, k=100)
    scores=preds.values.numpy()
    code = preds.indices.numpy()
    prediction = pd.DataFrame(
        {
            'session_id':session_ids,
            'code':[tuple(v) for v in code],
            'score':[tuple(v) for v in scores],
            'rank':[tuple([rank for rank in range(1, 101)]) for id in session_ids]
        }
        )

    submission_df = pd.concat([submission_df, prediction])

submission_df.head()

,session_id,code,score,rank
0,61.0,"(1045, 4574, 429, 3586, 1016, 1563, 3414, 2156...","(0.30723658, 0.029230941, 0.02872988, 0.018992...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
1,96.0,"(3564, 3778, 4148, 19, 1182, 3155, 2063, 3045,...","(0.080837496, 0.03586632, 0.024371354, 0.01645...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
2,185.0,"(4146, 4148, 2330, 1936, 3326, 1652, 3827, 733...","(0.35210347, 0.05719872, 0.046379857, 0.018238...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
3,224.0,"(1437, 2448, 4202, 4347, 4708, 16, 4418, 3423,...","(0.072553806, 0.049857028, 0.039590623, 0.0214...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."
4,285.0,"(3441, 1070, 3555, 2970, 3691, 2471, 4028, 545...","(0.056309663, 0.05243944, 0.04584098, 0.038928...","(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14..."


In [ ]:
submission_df.to_csv(
    './drive/MyDrive/Recommendation_system/dataset/candidates/NN/Transformer/final/candidates.csv',
    index=False
    )